# My First Convolutional Neural Network : Fashion MNIST
---
#### 실습목표
1. CNN의 기본 아이디어를 안다.
2. CNN의 구조를 그리고, 코드로 옮길 수 있다.

### Keras Upgrade

In [ ]:
!pip install keras-nightly

## Real Game : CNN on Fashion MNIST

여기에서는 여러분이 직접 코드를 완성해야 하는 문제가 곳곳에 있습니다!

In [ ]:
'''
라이브러리들을 불러오자.
'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random as rd
from sklearn.metrics import accuracy_score

import keras

### Data Loading

In [ ]:
(train_x, train_y), (test_x, test_y) = keras.datasets.fashion_mnist.load_data()

In [ ]:
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

In [ ]:
labels = ["T-shirt/top",  # index 0
            "Trouser",      # index 1
            "Pullover",     # index 2
            "Dress",        # index 3
            "Coat",         # index 4
            "Sandal",       # index 5
            "Shirt",        # index 6
            "Sneaker",      # index 7
            "Bag",          # index 8
            "Ankle boot"]   # index 9

print(labels)

In [ ]:
'''
Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

id = rd.randrange(0,10000)

print(f'id = {id}')
print(f'다음 그림은 {labels[test_y[id]]} 입니다.')

plt.imshow(test_x[id], cmap='gray')
plt.show()

### X : Min-Max Scaling

- 최소값 0, 최대값 1로 통일하는 스케일링

In [ ]:
max_v, min_v = train_x.max(), train_x.min()
max_v, min_v

In [ ]:
train_x = (train_x - min_v) / (max_v - min_v)
test_x = (test_x - min_v) / (max_v - min_v)

In [ ]:
train_x.max(), train_x.min()

### X : Reshape(# of data, 28, 28, 1)

- **끝에 1을 달아서 그레이스케일(흑백)을 명시해준다.**

In [ ]:
train_x.shape

In [ ]:
train_x = train_x.reshape(train_x.shape[0], 28, 28, -1)
test_x = test_x.reshape(test_x.shape[0], 28, 28, -1)

In [ ]:
train_x.shape

### Y : One-Hot Encoding

In [ ]:
train_y.shape

In [ ]:
len_y = len(np.unique(train_y))
len_y

In [ ]:
from keras.utils import to_categorical

In [ ]:
train_y = to_categorical(train_y, len_y)
test_y = to_categorical(test_y, len_y)

In [ ]:
train_x.shape, train_y.shape

### 코드를 완성해주세요!

- **자유롭게 먼저 해보는 것을 추천**
---
- **구조를 따라서 코딩을 한다면..**

    1. 인풋 레이어
    2. Convolution : 필터수 32개, 사이즈(3, 3), same padding
    3. BatchNormalization
    4. Convolution : 필터수 32개, 사이즈(3, 3), same padding
    5. BatchNormalization
    6. MaxPooling : 사이즈(2,2) 스트라이드(2,2)
    7. DropOut : 25% 비활성화
    8. Convolution : 필터수 64개, 사이즈(3, 3), same padding
    9. BatchNormalization
    10. Convolution : 필터수 64개, 사이즈(3, 3), same padding
    11. BatchNormalization
    12. MaxPooling : 사이즈(2,2) 스트라이드(2,2)
    13. DropOut : 25% 비활성화
    14. Flatten
    15. Fully Connected Layer : 노드 512개
    16. BatchNormalization
    17. 아웃풋 레이어

In [ ]:
import keras

from keras.backend import clear_session
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, BatchNormalization, Dropout, Conv2D, MaxPool2D

In [ ]:
clear_session()

model = Sequential()

##################################
# 인풋 레이어
model.add( Input(shape=(28,28,1)) )
# Convolution : 필터수 32개, 사이즈(3, 3), same padding
model.add( Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu') )
# BatchNormalization
model.add( BatchNormalization() )
# Convolution : 필터수 32개, 사이즈(3, 3), same padding
model.add( Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu') )
# BatchNormalization
model.add( BatchNormalization() )
# MaxPooling : 사이즈(2,2) 스트라이드(2,2)
model.add( MaxPool2D(pool_size=(2,2), strides=(2,2)) )
# DropOut : 25% 비활성화
model.add( Dropout(0.25) )
# Convolution : 필터수 64개, 사이즈(3, 3), same padding
model.add( Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu') )
# BatchNormalization
model.add( BatchNormalization() )
# Convolution : 필터수 64개, 사이즈(3, 3), same padding
model.add( Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu') )
# BatchNormalization
model.add( BatchNormalization() )
# MaxPooling : 사이즈(2,2) 스트라이드(2,2)
model.add( MaxPool2D(pool_size=(2,2), strides=(2,2)) )
# DropOut : 25% 비활성화
model.add( Dropout(0.25) )
# Flatten
model.add( Flatten() )
# Fully Connected Layer : 노드 512개
model.add( Dense(512, activation='relu') )
# BatchNormalization
model.add( BatchNormalization() )
# 아웃풋 레이어
model.add( Dense(10, activation='softmax') )
##################################

model.compile(loss=keras.losses.categorical_crossentropy, metrics=['accuracy'],
              optimizer='adam')

model.summary()

### Early Stopping을 활용한 학습

- 조건
    1. validation_split = 0.2
    2. 1 epochs만 관찰해가며 속도가 가장 빠른 batch_size 찾아보기. 128개부터 시작하여 조절해볼 것.
    3. EarlyStopping. val_loss가 5 epoch 전과 비교하여 개선되지 않으면 스탑
    4. 이미지 시각화를 위해 hist 변수에 학습 과정을 넣어주세요.


In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=5,
                   verbose=1,
                   restore_best_weights=True)

In [ ]:
hist = model.fit(train_x, train_y, epochs=10000, verbose=1,
                 validation_split=0.2, callbacks=[es], batch_size=64)

### 이미지 시각화

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy : Training vs Validation')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc=0)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss : Training vs Validation')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc=0)
plt.show()

In [ ]:
y_pred = model.predict(test_x)
y_pred.shape

In [ ]:
single_y_pred = y_pred.argmax(axis=1)
single_y_pred.shape

In [ ]:
print(test_y.shape)
single_test_y = test_y.argmax(axis=1)
print(single_test_y.shape)

In [ ]:
test_acc = accuracy_score(single_test_y, single_y_pred)

print(f'테스트셋 정확도 : {test_acc*100:.2f}%' )

In [ ]:
'''
성능 확인을 위해
Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

id = rd.randrange(0,10000)

print(f'id = {id}')
print(f'다음 그림은 숫자 {labels[single_test_y[id]]} 입니다.')
print(f'모델의 예측 : {labels[single_y_pred[id]]}')
print(f'모델의 카테고리별 확률 : {np.floor(y_pred[id]*100)}')

if single_test_y[id] == single_y_pred[id] :
    print('정답입니다')
else :
    print('틀렸어요')

plt.imshow(test_x[id].reshape([28,-1]), cmap='gray')
plt.show()

In [ ]:
'''
틀린 것만 관찰해보자!

Ctrl+Enter를 이용하여
반복 실행 해보자!
'''

true_false = (single_test_y==single_y_pred)
f_id = np.where(true_false==False)[0]        # 틀린 것들의 인덱스 반환
f_n = len(f_id)                              # 틀린 것들이 총 몇 개나 될까?

id = f_id[rd.randrange(0,f_n)]

print(f'id = {id}')
print(f'다음 그림은 숫자 {labels[single_test_y[id]]} 입니다.')
print(f'모델의 예측 : {labels[single_y_pred[id]]}')
print(f'모델의 카테고리별 확률 : {np.floor(y_pred[id]*100)}')

if single_test_y[id] == single_y_pred[id] :
    print('정답입니다')
else :
    print('틀렸어요')

plt.imshow(test_x[id].reshape([28,-1]), cmap='gray')
plt.show()